# Multi-Mission MAG

> Abstract: An introduction to "the greater Swarm": calibrated platform magnetometer data from other missions. We begin with Cryosat-2, GRACE, and GRACE-FO.

See also:
- https://nbviewer.jupyter.org/github/pacesm/jupyter_notebooks/blob/master/MAG/MAG_multi-mission_CHAOS_residuals_demo.ipynb

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib,cartopy

In [ ]:
from viresclient import SwarmRequest
import datetime as dt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from tqdm.notebook import tqdm

## Product information

Platform magnetometer data from other missions have been carefully recalibrated so that they are accurate and suitable for usage in geomagnetic field modelling. The data currently available from VirES are as follows:

- CryoSat-2:  
  Olsen, N., Albini, G., Bouffard, J. et al. Magnetic observations from CryoSat-2: calibration and processing of satellite platform magnetometer data. Earth Planets Space 72, 48 (2020).  
  <https://doi.org/10.1186/s40623-020-01171-9>  
  VirES collection names: `"CS_OPER_MAG"`
- GRACE (x2):  
  Olsen, N. Magnetometer data from the GRACE satellite duo. Earth Planets Space 73, 62 (2021).  
  <https://doi.org/10.1186/s40623-021-01373-9>  
  VirES collection names: `"GRACE_A_MAG"`, `"GRACE_B_MAG"`  
- GRACE-FO (x2):  
  Stolle, C., Michaelis, I., Xiong, C. et al. Observing Earth’s magnetic environment with the GRACE-FO mission. Earth Planets Space 73, 51 (2021).  
  <https://doi.org/10.1186/s40623-021-01364-w>  
  VirES collection names: `"GF1_OPER_FGM_ACAL_CORR"`, `"GF2_OPER_FGM_ACAL_CORR"`
  
The variables available from each collection are:

In [ ]:
request = SwarmRequest()
for collection in ("CS_OPER_MAG", "GRACE_A_MAG", "GF1_OPER_FGM_ACAL_CORR"):
    print(f"{collection}:\n{request.available_measurements(collection)}\n")

Where additional `B_NEC` variables are specified (`B_NEC1`, `B_NEC2`, `B_NEC3`), these correspond to measurements from separate magnetometers on-board the spacecraft. See the scientific publications for details. Magnetic model evaluation will also work with those variables.

The temporal availabilities of data are:

In [ ]:
for collection in (
        "CS_OPER_MAG",
        "GRACE_A_MAG", "GRACE_B_MAG",
        "GF1_OPER_FGM_ACAL_CORR", "GF2_OPER_FGM_ACAL_CORR"
    ):
    df = request.available_times(collection)
    start = df["starttime"].iloc[0]
    end = df["endtime"].iloc[-1]
    print(f"{collection}:\nAvailability: {start} --- {end}\n")

## Access works just like Swarm MAG products

We can specify which collection to fetch, and which models to evaluate at the same time:

In [ ]:
request = SwarmRequest()
request.set_collection("GF1_OPER_FGM_ACAL_CORR")
request.set_products(["B_NEC"], models=["IGRF"])
data = request.get_between("2018-06-01", "2018-06-02")
ds = data.as_xarray()
ds

In [ ]:
# Append the residual, B - IGRF
ds["B_NEC_res_IGRF"] = ds["B_NEC"] - ds["B_NEC_IGRF"]
# Plot (B) and (B - IGRF) to compare
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5), sharex=True)
ds["B_NEC"].plot.line(x="Timestamp", ax=axes[0])
ds["B_NEC_res_IGRF"].plot.line(x="Timestamp", ax=axes[1]);

## Data from multiple spacecraft

Here is an example of fetching and visualising data from multiple spacecraft. We select a day where we can get data from Swarm, CryoSat, and GRACE-FO.

In [ ]:
START = dt.datetime(2018, 6, 1)
END = dt.datetime(2018, 6, 2)

In [ ]:
# Mappings to identify spacecraft and collection names
# Let's disable Swarm Charlie & GRACE-FO 2 for now
#   as they are in similar places as Swarm Alpha and GRACE-FO 1
spacecraft_to_collections = {
    "Swarm Alpha": "SW_OPER_MAGA_LR_1B",
    "Swarm Bravo": "SW_OPER_MAGB_LR_1B",
#     "Swarm Charlie": "SW_OPER_MAGC_LR_1B",
    "CryoSat-2": "CS_OPER_MAG",
    "GRACE-FO 1": "GF1_OPER_FGM_ACAL_CORR",
#     "GRACE-FO 2": "GF2_OPER_FGM_ACAL_CORR"
}
collections_to_spacecraft = {v: k for k, v in spacecraft_to_collections.items()}

def fetch_sc(sc_collection, start_time=START, end_time=END, **kwargs):
    """Fetch data from a specific spacecraft"""
    request = SwarmRequest()
    request.set_collection(sc_collection)
    request.set_products(["B_NEC"])
    data = request.get_between(start_time, end_time, **kwargs)
    ds = data.as_xarray()
    # Rename the Spacecraft variable to use the mission name too
    ds.Spacecraft[:] = collections_to_spacecraft[sc_collection]
    return ds

ds_set = {}
for sc in tqdm(spacecraft_to_collections.keys()):
    collection = spacecraft_to_collections[sc]
    ds_set[sc] = fetch_sc(collection, asynchronous=False, show_progress=False)

Data are now stored within datasets within a dictionary:

In [ ]:
ds_set["Swarm Alpha"]

A quick inspection of the data:

In [ ]:
ds_set["Swarm Alpha"].plot.scatter(x="Longitude", y="Latitude", hue="B_NEC", s=0.1);

In [ ]:
fig, axes = plt.subplots(
    nrows=2, figsize=(15,15),
    subplot_kw={"projection": ccrs.PlateCarree()}
)
for ax in axes:
    ax.add_feature(cfeature.COASTLINE, edgecolor='silver')
for sc in ("Swarm Alpha", "Swarm Bravo", "CryoSat-2", "GRACE-FO 1"):
    # Extract dataset and plot contents
    _ds = ds_set[sc]
    lon, lat = _ds["Longitude"], _ds["Latitude"]
    B_C = _ds["B_NEC"].sel(NEC="C").values
    # Plot positions coloured by spacecraft
    axes[0].scatter(x=lon, y=lat, s=0.1, label=sc)
    norm = plt.Normalize(vmin=-60000, vmax=60000)
    cmap = "viridis"
    # Plot 
    axes[1].scatter(x=lon, y=lat, c=B_C, s=0.1, norm=norm, cmap=cmap)
fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes[1], label=r"$B_C$ [nT]");
axes[0].set_title("Orbits from each spacecraft")
axes[1].set_title("Vertical component magnetic measurements");